In [33]:
import numpy as np

In [46]:
# basic parameters:
arrival_rate = 0.8
background_speed = 12 # 25mph = 11.176 m/s
cruising_speed = 0.25 * background_speed
segment_length = 1000
vehicle_length = 4.5 # a typical vehicle length is 4.5 meters
pull_over_dist = 3 # The artifical pull over distance is 3 m


# Generate possion arrival for left land and right lane as the inter-arrival time is following exponential distribution
beta = 1 / arrival_rate
left_interarr_times = np.random.exponential(scale = beta, size = 1000)
right_interarr_times = np.random.exponential(scale = beta, size = 1000)

# So the inter-vehicle distances could be calculated as time * background_speed
left_interarr_dists = left_interarr_times * background_speed
right_interarr_dists = right_interarr_times * background_speed

# The physical stamp is the distance between of the fronts of two vehicle
# To make sure that there is at least a vehicle, we need to filter those distance < 1.1 * vehicle_length and replace with 1.1 * vehicle_length
left_interarr_dists_modified = np.clip(left_interarr_dists, 1.1 * vehicle_length, np.inf)
right_interarr_dists_modified = np.clip(right_interarr_dists, 1.1 * vehicle_length, np.inf)

# Get the coordinates from the intersection of the front of each vehicles, which is the cumulative of distances between vehicles:
left_vehicle_coordinates = np.cumsum(left_interarr_dists_modified, dtype = float)
left_vehicle_coordinates = left_vehicle_coordinates[left_vehicle_coordinates < segment_length]
right_vehicle_coordinates = np.cumsum(right_interarr_dists_modified, dtype = float)
right_vehicle_coordinates = right_vehicle_coordinates[right_vehicle_coordinates < segment_length]

In [47]:
# required length for pull over slot:
required_length = 2 * vehicle_length

In [48]:
# Lets look again at the left vehicle coordinates of the vehicle on the left lane:
left_vehicle_coordinates

array([  4.95      ,   9.9       ,  14.85      ,  19.90371992,
        49.3604563 ,  54.3104563 ,  60.53194386,  65.48194386,
        98.28918518, 108.27445005, 113.22445005, 119.3097612 ,
       124.2597612 , 145.82137631, 153.80304742, 158.75304742,
       163.70304742, 174.26931731, 206.58200874, 218.44360285,
       223.39360285, 263.51219086, 313.06590412, 318.01590412,
       322.96590412, 339.68207931, 355.66139949, 370.69908296,
       375.64908296, 402.43319125, 408.5427021 , 455.74666319,
       487.89820886, 499.54649319, 509.37411117, 516.95678312,
       523.12358225, 531.32348105, 537.24607781, 543.29892672,
       559.49911718, 569.79570229, 577.26213045, 582.21213045,
       609.0066817 , 619.96634353, 624.91634353, 629.86634353,
       653.43914229, 663.69197345, 670.68149739, 680.18536976,
       687.769955  , 701.42380782, 706.41559856, 718.30367806,
       769.75192721, 776.89845249, 791.64497114, 806.31671152,
       816.405277  , 826.02156003, 835.79578483, 865.17

In [49]:
# And we can get the number of the vehicle in front of Emergency Vehicle of interest:
left_vehicle_coordinates.shape

(72,)

In [50]:
# Lets look at the coordinates of the vehicles on the right lane:
right_vehicle_coordinates

array([ 35.14605333,  40.48750638,  45.43750638, 105.87425231,
       110.82425231, 125.91733624, 166.34494855, 171.29494855,
       180.19651786, 185.14651786, 191.31685846, 215.80442166,
       221.88137654, 233.59207577, 250.99085418, 255.94085418,
       269.76344349, 297.39515842, 302.34515842, 307.29515842,
       339.72310435, 348.27638143, 353.22638143, 359.18478908,
       369.63901032, 383.9051427 , 468.41702065, 503.55549876,
       531.91201914, 536.86201914, 576.44715782, 581.39715782,
       586.34715782, 595.92744226, 600.87744226, 631.88700971,
       637.94691486, 642.89691486, 696.70594943, 727.78461181,
       732.73461181, 741.57090586, 747.80902477, 756.2442479 ,
       766.36726804, 771.80912217, 776.75912217, 782.27967361,
       792.98043778, 797.93043778, 809.77626467, 816.98029941,
       845.57318964, 850.52318964, 855.47318964, 860.57607782,
       865.52607782, 877.79896843, 882.74896843, 887.69896843,
       892.88459217, 977.30191596])

In [51]:
# The number of vehicles on the right lane is:
len(right_vehicle_coordinates)

62

In [132]:
# We then have to calculate the exact coordinates of those slots, starting from 0:
empty_slots = []
current_position = 0
for i in range(len(right_vehicle_coordinates)):
    head = current_position
    tail = right_vehicle_coordinates[i]
    empty_slots.append([head, tail])
    current_position = tail + vehicle_length
if (current_position - vehicle_length) < segment_length :
    empty_slots.append([current_position, segment_length])

In [133]:
# And we could know that there should be number_vehicle + 1 slots for possible pulling over:
len(empty_slots)

63

In [134]:
# Pull over rules for distributed systems:
# 1. a vehicle could only pull over to a spot that is physically ahead of him, which means the head of this slot
# should be smaller than the front of the vehicle;
# 2. the empty slot should at least be 2 * vehicle length long
# 3. Since there is a pull over distance, the back of the vehicle - pull over distance should be smaller than 
# the tail of the slot
def can_pull_over(vehicle_coordinate, slot):
    head = slot[0]
    tail = slot[1]
    vehicle_back = vehicle_coordinate + vehicle_length
    if (tail - head) < required_length:
        return False
    elif vehicle_coordinate - pull_over_dist < head:
        return False
    return True

In [135]:
# Then we can see which slots a vehicle can pull over into, if there is no available slots, then it will exit from the intersection
# For example, lets take a look at the first vehicle:
first_veh_coordinate = left_vehicle_coordinates[0]
first_veh_coordinate

4.95

In [136]:
for slot_element in empty_slots:
    result = can_pull_over(first_veh_coordinate, slot_element)
    if result:
        print(slot_element)
    else:
        break

[0, 35.14605332759829]


In [137]:
# A special rule regarding the distributed system is that all vehicles will try to pull over into their nearest
# slot. However, with respect to time, we might not know which vehicle would pull in first.
# So, we need to calculate that and pick the smallest one to be timestamp of interest. Through the process, vehicles
# to be cleared and empty slots are changing per timestamp of interest

In [138]:
# First of all, we need to know which slot each vehicle is going to pull over into at timestamp 0:
def slot_to_park_distributed(vehicle_coordinate, empty_slots):
    # default is exiting at the intersection
    res = 0
    
    for slot_element in empty_slots:
        if can_pull_over(vehicle_coordinate, slot_element):
            res = slot_element     
    return res

In [139]:
# Check if all satisfy the pull over slots:
for vehicle_coordinate in left_vehicle_coordinates:
    print(vehicle_coordinate, slot_to_park_distributed(vehicle_coordinate, empty_slots))

4.95 [0, 35.14605332759829]
9.9 [0, 35.14605332759829]
14.850000000000001 [0, 35.14605332759829]
19.903719917266876 [0, 35.14605332759829]
49.360456300157644 [0, 35.14605332759829]
54.31045630015765 [49.937506376145265, 105.87425230850936]
60.53194386413139 [49.937506376145265, 105.87425230850936]
65.4819438641314 [49.937506376145265, 105.87425230850936]
98.28918518328454 [49.937506376145265, 105.87425230850936]
108.27445004906419 [49.937506376145265, 105.87425230850936]
113.22445004906419 [49.937506376145265, 105.87425230850936]
119.30976119736331 [115.32425230850936, 125.91733624142336]
124.25976119736332 [115.32425230850936, 125.91733624142336]
145.82137631433528 [130.41733624142336, 166.34494854560091]
153.8030474201564 [130.41733624142336, 166.34494854560091]
158.75304742015638 [130.41733624142336, 166.34494854560091]
163.70304742015637 [130.41733624142336, 166.34494854560091]
174.26931730726986 [130.41733624142336, 166.34494854560091]
206.5820087423697 [195.8168584622593, 215.804

In [146]:
# Compute the smallest pull over time at certain timestamp:
def next_timestamp(vehicle_coordinates, empty_slots):
    pull_over_times = []
    for vehicle in vehicle_coordinates:
        
        vehicle_back = vehicle + vehicle_length
        
        tail = slot_to_park_distributed(vehicle, empty_slots)[1]
        
        if tail <= vehicle_back - pull_over_dist:
            pull_over_time = (vehicle_back - tail) / cruising_speed
        else:
            pull_over_time = pull_over_dist / cruising_speed
        pull_over_times.append(pull_over_time)
    
    Min = np.min(pull_over_times)
    min_positions = [i for i, x in enumerate(pull_over_times) if x == Min]
    return Min, min_positions

In [147]:
next_timestamp(left_vehicle_coordinates, empty_slots)

(1.0,
 [0,
  1,
  2,
  3,
  5,
  6,
  7,
  8,
  11,
  12,
  13,
  14,
  15,
  16,
  18,
  21,
  23,
  24,
  29,
  30,
  31,
  32,
  33,
  35,
  36,
  40,
  41,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  54,
  55,
  61,
  62,
  67,
  68,
  69,
  71])

In [ ]:
# We need to start the iteration, 